In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("all_kindle_review.csv")

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
df=df[["reviewText","rating"]]

In [5]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [6]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [7]:
df["rating"].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [8]:
df["rating"].value_counts()

5    3000
4    3000
3    2000
2    2000
1    2000
Name: rating, dtype: int64

In [9]:
## preprocessing and cleaning

In [10]:
## if rating is above 3 positive review and consider as 1 below 3 rating negative review consider as 0
df["rating"]=df["rating"].apply(lambda x:0 if x<3 else 1)

In [11]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [12]:
df["rating"].unique()

array([1, 0], dtype=int64)

In [13]:
df["rating"].value_counts()

1    8000
0    4000
Name: rating, dtype: int64

In [15]:
##lower all the cases
df["reviewText"]=df["reviewText"].str.lower()

In [17]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [18]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\2252288\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from bs4 import BeautifulSoup

In [20]:
## removing special characters
df["reviewText"]=df["reviewText"].apply(lambda x:re.sub("[^a-z A-Z 0-9]+","",x))
## remove the stopwords
df["reviewText"]=df["reviewText"].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words("english")]))
## remove url
df["reviewText"]=df["reviewText"].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\\w_-]+(?:(?:\\.[\\w_-]+)+))([\\w.,@?^=%&:/~+#-]*[\\w@?^=%&/~+#-])?',"",str(x)))
## remove html tags
df["reviewText"]=df["reviewText"].apply(lambda x:BeautifulSoup(x,"lxml").get_text() )
## remove additional spaces
df["reviewText"]=df["reviewText"].apply(lambda x:" ".join(x.split()))

In [21]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [22]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [27]:
lemmatizer=WordNetLemmatizer()

In [30]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    

In [31]:
df["reviewText"]=df["reviewText"].apply(lambda x:lemmatize_words(x))

In [32]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [35]:
## train test split
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df["reviewText"],df["rating"], test_size=0.33, random_state=42)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [38]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [39]:
from sklearn.naive_bayes import GaussianNB
nb_model=GaussianNB().fit(X_train_bow,y_train)

In [40]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [41]:
y_pred_bow=nb_model.predict(X_test_bow)

In [43]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.591919191919192


In [44]:
confusion_matrix(y_test,y_pred_bow)

array([[ 852,  506],
       [1110, 1492]], dtype=int64)

## word2vec

In [48]:
## one more method for data preprocessing
from nltk import sent_tokenize
from gensim.utils import simple_preprocess    

In [50]:
words=[]
for sent in df["reviewText"]:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [51]:
words

[['jace',
  'rankin',
  'may',
  'short',
  'he',
  'nothing',
  'mess',
  'man',
  'hauled',
  'saloon',
  'undertaker',
  'know',
  'he',
  'famous',
  'bounty',
  'hunter',
  'oregon',
  'shot',
  'man',
  'saloon',
  'finished',
  'year',
  'long',
  'quest',
  'avenge',
  'sister',
  'murder',
  'trying',
  'figure',
  'next',
  'snottynosed',
  'farm',
  'boy',
  'rescued',
  'gang',
  'bully',
  'offer',
  'money',
  'kill',
  'man',
  'forced',
  'ranch',
  'reluctantly',
  'agrees',
  'bring',
  'man',
  'justice',
  'kill',
  'outright',
  'first',
  'need',
  'tell',
  'sister',
  'widower',
  'newskyla',
  'kyle',
  'springer',
  'bailey',
  'riding',
  'trail',
  'sleeping',
  'ground',
  'past',
  'month',
  'trying',
  'find',
  'jace',
  'want',
  'revenge',
  'man',
  'killed',
  'husband',
  'took',
  'ranch',
  'amongst',
  'crime',
  'shes',
  'keen',
  'detour',
  'jace',
  'want',
  'take',
  'realizes',
  'shes',
  'option',
  'hide',
  'behind',
  'boy',
  'pers

In [45]:
import gensim
import numpy as np

In [57]:
## lets train word2vec from scratch
model=gensim.models.Word2Vec(words,epochs=10)

In [58]:
## To get all the vocabulary
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'much',
 'first',
 'well',
 'even',
 'didnt',
 'short',
 'know',
 'way',
 'could',
 'great',
 'make',
 'sex',
 'little',
 'dont',
 'thing',
 'want',
 'two',
 'think',
 'find',
 'plot',
 'also',
 'romance',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'take',
 'woman',
 'many',
 'lot',
 'written',
 'kindle',
 'say',
 'work',
 'thought',
 'bit',
 'found',
 'year',
 'going',
 'give',
 'interesting',
 'liked',
 'writing',
 'novel',
 'loved',
 'another',
 'feel',
 'better',
 'got',
 'come',
 'man',
 'hot',
 'still',
 'enough',
 'back',
 'though',
 'people',
 'reader',
 'made',
 'star',
 'review',
 'something',
 'part',
 'friend',
 'cant',
 'page',
 'need',
 'bad',
 'keep',
 'new',
 'wasnt',
 'free',
 'world',
 'doesnt',
 'relationship',
 'enjoy',
 'recommend',
 'together',
 'next',
 'start',
 'felt',
 'best',
 'put',

In [59]:
model.corpus_count

12000

In [60]:
model.epochs

10

In [61]:
model.wv.similar_by_word("good")

[('great', 0.7450907826423645),
 ('decent', 0.7139726877212524),
 ('amazing', 0.5869136452674866),
 ('nice', 0.5868203043937683),
 ('excellent', 0.5669181942939758),
 ('solid', 0.5616794228553772),
 ('fantastic', 0.5540508031845093),
 ('fine', 0.551551103591919),
 ('exciting', 0.5458858609199524),
 ('awesome', 0.5311671495437622)]

In [62]:
words[0]

['jace',
 'rankin',
 'may',
 'short',
 'he',
 'nothing',
 'mess',
 'man',
 'hauled',
 'saloon',
 'undertaker',
 'know',
 'he',
 'famous',
 'bounty',
 'hunter',
 'oregon',
 'shot',
 'man',
 'saloon',
 'finished',
 'year',
 'long',
 'quest',
 'avenge',
 'sister',
 'murder',
 'trying',
 'figure',
 'next',
 'snottynosed',
 'farm',
 'boy',
 'rescued',
 'gang',
 'bully',
 'offer',
 'money',
 'kill',
 'man',
 'forced',
 'ranch',
 'reluctantly',
 'agrees',
 'bring',
 'man',
 'justice',
 'kill',
 'outright',
 'first',
 'need',
 'tell',
 'sister',
 'widower',
 'newskyla',
 'kyle',
 'springer',
 'bailey',
 'riding',
 'trail',
 'sleeping',
 'ground',
 'past',
 'month',
 'trying',
 'find',
 'jace',
 'want',
 'revenge',
 'man',
 'killed',
 'husband',
 'took',
 'ranch',
 'amongst',
 'crime',
 'shes',
 'keen',
 'detour',
 'jace',
 'want',
 'take',
 'realizes',
 'shes',
 'option',
 'hide',
 'behind',
 'boy',
 'persona',
 'best',
 'try',
 'keep',
 'pace',
 'confrontation',
 'along',
 'way',
 'get',
 'sh

In [63]:
def avg_word2vec(doc):
    # remove out of vocabulary words 
    # sent = [word for word in doc if word in model.wv.index_to_key]
    # print(sent)
    word_vectore=[model.wv[word] for word in doc if word in model.wv.index_to_key]
    return np.mean(word_vectore,axis=0)

In [64]:
from tqdm import tqdm

In [65]:
# apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 12000/12000 [00:20<00:00, 575.83it/s]


In [66]:
X

[array([-0.07765795,  0.26921925,  0.13848196,  0.25534764, -0.01627725,
        -0.17201184, -0.04016847,  0.42507267, -0.17594238, -0.08363478,
        -0.02027852, -0.07666249,  0.21377735,  0.07336345,  0.11963312,
        -0.19033806,  0.04999053, -0.44527942,  0.07075815, -0.31130713,
        -0.03174245, -0.17703806, -0.00152243, -0.04703251,  0.26383114,
        -0.02567503, -0.33387393,  0.06016822, -0.18817878,  0.06616446,
         0.3355418 , -0.19486639,  0.05305384, -0.19624205,  0.01465982,
         0.2615854 ,  0.0643663 , -0.29067084, -0.20084175, -0.49231157,
        -0.21319921, -0.20091233, -0.14240809, -0.02442919,  0.1736423 ,
        -0.2027122 , -0.25126088, -0.11981604,  0.1006052 ,  0.14902554,
         0.21385989, -0.33434537, -0.18716493,  0.27711737, -0.29680347,
         0.18813993,  0.05293484, -0.09552507, -0.44933546, -0.02774589,
        -0.13321869,  0.05782755, -0.1648706 , -0.04575722, -0.22018713,
         0.29120162,  0.15802999,  0.19343753, -0.4

In [67]:
len(X)

12000

In [68]:
##independent features
X_new = np.array(X)

In [71]:
X_new.shape

(12000, 100)

In [74]:
##Dependent features
#y=messages[list(map(lambda x: len(x) >0,corpus))]# to remove missing values
#y=pd.get_dummies(y["rating"])
#y=y.iloc[:,0].values

y=df["rating"]

In [76]:
## final independent features
df=pd.DataFrame()
for i in range(0,len(X)):
    df=df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)

C:\Users\2252288\AppData\Local\Temp\ipykernel_18680\2009573435.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)


In [77]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.077658,0.269219,0.138482,0.255348,-0.016277,-0.172012,-0.040168,0.425073,-0.175942,-0.083635,...,0.291450,0.264073,-0.224664,0.253780,0.348239,0.110400,-0.050669,-0.219038,-0.019482,-0.156726
1,-0.136426,-0.144905,0.344061,0.535808,0.242992,-0.347705,0.502045,0.515771,-0.532789,-0.218411,...,0.652539,0.515128,0.069105,0.800817,-0.141941,0.311371,0.158175,-0.465038,-0.070496,-0.292241
2,-0.182112,0.037866,0.376999,0.242446,0.176353,-0.236363,0.337443,0.491284,-0.361948,-0.189117,...,0.476697,0.361878,0.126021,0.405805,0.149108,0.396251,-0.028387,-0.344424,-0.173405,-0.061513
3,-0.219691,-0.052447,0.413093,0.061614,0.117353,-0.247661,0.154908,0.341267,-0.345923,-0.123974,...,0.160174,-0.073464,0.317606,0.078545,-0.085112,0.240475,0.404712,-0.317036,-0.287069,-0.432548
4,0.318310,0.337096,-0.229038,-0.083788,-0.285043,-0.385726,0.529492,0.866368,-0.263605,-0.071103,...,0.456167,0.293001,0.905690,0.382283,0.447075,0.411464,0.259037,-0.580888,-0.319396,-0.183503


In [79]:
X=df

In [80]:
X.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    0
Length: 100, dtype: int64

In [81]:
y

0        1
1        1
2        1
3        1
4        1
        ..
11995    1
11996    1
11997    1
11998    0
11999    1
Name: rating, Length: 12000, dtype: int64

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [85]:
from sklearn.naive_bayes import GaussianNB
nb_model=GaussianNB().fit(X_train,y_train)

In [86]:
y_pred=nb_model.predict(X_test)

In [87]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [88]:
print(accuracy_score(y_test,y_pred))

0.7522727272727273


In [89]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.76      0.68      1358
           1       0.86      0.75      0.80      2602

    accuracy                           0.75      3960
   macro avg       0.73      0.75      0.74      3960
weighted avg       0.77      0.75      0.76      3960



In [91]:
print(confusion_matrix(y_test,y_pred))

[[1035  323]
 [ 658 1944]]
